## ⚗️ Diseño Óptimo de un Reactor Químico (PFR)

Se desea encontrar la combinación óptima de **temperatura ($T$)** y **volumen ($V$)** de un reactor de flujo pistón continuo (PFR) que **maximice la conversión $X$** del reactivo A.

---

### 🔸 Datos del problema

| Parámetro | Valor | Descripción |
|----------|-------|-------------|
| $C_{A0}$ | 1.0 mol/L | Concentración inicial de A |
| $F_{A0}$ | 10 mol/min | Flujo molar de entrada de A |
| $k_0$ | $1.0 \times 10^6$ 1/min | Factor pre-exponencial |
| $E$ | 8000 cal/mol | Energía de activación |
| $R$ | 1.987 cal/mol·K | Constante de los gases |
| $T$ | Variable (300–500 K) | Temperatura |
| $V$ | Variable (0–10 m³) | Volumen del reactor |

---

### 🔸 Modelo Matemático

1. **Constante de velocidad de reacción (Ley de Arrhenius):**

$$
k = k_0 \cdot \exp\left(-\frac{E}{R T}\right)
$$

2. **Conversión fraccional del reactivo:**

$$
X = 1 - \exp\left(-\frac{k \cdot V}{F_{A0}}\right)
$$

3. **Función objetivo:**

Maximizar $X$, es decir:

$$
\max_{T, V} \left[ 1 - \exp\left( -\frac{k_0 \cdot \exp\left(-\frac{E}{R T}\right) \cdot V}{F_{A0}} \right) \right]
$$

4. **Restricciones:**

$$
300 \leq T \leq 500 \\
0 \leq V \leq 10
$$

---

### 💻 Implementación en Pyomo (Optimización No Lineal)

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Crear el modelo
model = ConcreteModel()

# Parámetros
CA0 = 1.0       # mol/L
FA0 = 10.0      # mol/min
k0 = 1.0e6      # 1/min
E = 8000.0      # cal/mol
R = 1.987       # cal/mol·K

# Variables de decisión
model.T = Var(bounds=(300, 500))     # Temperatura (K)
model.V = Var(bounds=(0, 10))        # Volumen (m^3)

# Expresión para la constante de velocidad k
def k_rule(m):
    return k0 * exp(-E / (R * m.T))
model.k = Expression(rule=k_rule)

# Expresión para la conversión X
def X_rule(m):
    return 1 - exp(-m.k * m.V / FA0)
model.X = Expression(rule=X_rule)

# Objetivo: maximizar X
model.obj = Objective(expr=model.X, sense=maximize)

# Resolver el modelo con un solucionador de NLP (como IPOPT)
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Resultados
print("Temperatura óptima (K):", value(model.T))
print("Volumen óptimo (m^3):", value(model.V))
print("Conversión máxima:", value(model.X))

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Tot